In [1]:
import pandas as pd
import numpy as np
import torch
from argparse import Namespace
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Add the parent directory of 'ml' to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import warnings
warnings.filterwarnings('ignore')

from ml.utils.data_utils import prepare_dataset
from ml.models.lstm import LSTM
from ml.models.multi_step_lstm import MultiStepLSTM
from ml.models.seq2seq_lstm import Seq2SeqLSTM
from ml.models.transformer import TimeSeriesTransformer

In [2]:
# ===============================
# 0) CONFIG
# ===============================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TARGETS = ['rnti_count', 'rb_down', 'rb_up', 'down', 'up']
H = 6  # forecast_steps

# checkpoints
CKPT_BASE_T1   = "base_lstm_t1.pt"
CKPT_MULTI     = "multi_step_lstm.pt"
CKPT_S2S_MULTI = "seq2seq_lstm_multistep.pth"
CKPT_S2S_CLU   = "seq2seq_cluster_huber.pt"
CKPT_TRANS     = "transformer_multistep.pt"

In [3]:
# ===============================
# 1) DATA (full dataset) for 4 models
# ===============================
args_full = Namespace(
    data_path='../dataset/full_dataset.csv',
    targets=TARGETS,
    num_lags=10,
    forecast_steps=H,
    test_size=0.2,
    ignore_cols=None,
    identifier='District',
    nan_constant=0,
    x_scaler='minmax',
    y_scaler='minmax',
    outlier_detection=True,
    batch_size=128,
    cuda=torch.cuda.is_available(),
    seed=42
)

X_train, y_train, X_test, y_test, x_scaler_full, y_scaler_full, *_ = prepare_dataset(args_full)
# t+1 ground-truth on SCALED space (for Strategy A)
y_test_t1_scaled_full = y_test[:, 0, :]  # [N, 5]

N, L, D = X_test.shape
T = y_test.shape[2]


In [4]:
# ===============================
# 2) DATA (cluster dataset) for clustered Seq2Seq
# ===============================
args_cluster = Namespace(
    data_path='../dataset/combined_with_cluster_feature.csv',
    targets=TARGETS,
    num_lags=10,
    forecast_steps=H,
    test_size=0.2,
    ignore_cols=None,
    identifier='District',
    nan_constant=0,
    x_scaler='minmax',
    y_scaler='minmax',
    outlier_detection=True,
    batch_size=128,
    cuda=torch.cuda.is_available(),
    seed=42,
    use_time_features=False
)

X_tr_c, y_tr_c, X_te_c, y_te_c, x_scaler_c, y_scaler_c, *_ = prepare_dataset(args_cluster)
# t+1 ground-truth on SCALED space for cluster model
y_te_c_t1_scaled = y_te_c[:, 0, :]

In [5]:
# ===============================
# 3) HELPERS 
# ===============================
# ---------- core metrics on SCALED space (expects 2D arrays) ----------
def metrics_scaled_space(y_true_scaled_2d, y_pred_scaled_2d):
    mse  = mean_squared_error(y_true_scaled_2d, y_pred_scaled_2d)
    rmse = mean_squared_error(y_true_scaled_2d, y_pred_scaled_2d, squared=False)
    mae  = mean_absolute_error(y_true_scaled_2d, y_pred_scaled_2d)
    r2   = r2_score(y_true_scaled_2d, y_pred_scaled_2d)
    nrmse = rmse / (np.max(y_true_scaled_2d) - np.min(y_true_scaled_2d) + 1e-8)
    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2, "NRMSE": nrmse}

# ---------- percent metrics on ORIGINAL space (1 target column) ----------
def mape(y_true, y_pred, eps=1e-8):
    denom = np.clip(np.abs(y_true), eps, None)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100.0)

def smape(y_true, y_pred, eps=1e-8):
    denom = np.clip((np.abs(y_true) + np.abs(y_pred)) / 2.0, eps, None)
    return float(np.mean(np.abs(y_true - y_pred) / denom) * 100.0)

def masked_mape(y_true, y_pred, mask_thresh=1e-6, eps=1e-8):
    mask = np.abs(y_true) >= mask_thresh
    if mask.sum() == 0:
        return float('nan')
    denom = np.clip(np.abs(y_true[mask]), eps, None)
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / denom)) * 100.0)

def clamp_nonneg(x):
    return np.maximum(x, 0.0)

def inverse_single_col(y_scaled_1d, scaler, j):
    """
    Inverse-transform one target column j using the fitted MinMaxScaler 'scaler'.
    y_scaled_1d: shape [N,]
    """
    # Works for sklearn MinMaxScaler
    min_j   = np.asarray(scaler.min_)[j]
    scale_j = np.asarray(scaler.scale_)[j]
    return (y_scaled_1d - min_j) / scale_j

def percent_metrics_original_space_single(y_true_scaled_1d, y_pred_scaled_1d, scaler, j, clamp=True):
    """
    Inverse only column j back to ORIGINAL scale and compute percent metrics there.
    """
    yt = inverse_single_col(np.asarray(y_true_scaled_1d), scaler, j)
    yp = inverse_single_col(np.asarray(y_pred_scaled_1d), scaler, j)
    if clamp:
        yp = clamp_nonneg(yp)
    return {
        "MAPE%":        mape(yt, yp),
        "sMAPE%":       smape(yt, yp),
        "MAPE_masked%": masked_mape(yt, yp, mask_thresh=1e-6),
    }
    
def add_rows_for_model(name, y_pred_scaled, y_true_scaled, scaler, targets):
    for i, var in enumerate(targets):
        # per-target arrays on SCALED space (for core metrics)
        yt_s = y_true_scaled[:, i]
        yp_s = y_pred_scaled[:, i]

        # 2D for sklearn core metrics (still on scaled space)
        yt_s_2d = yt_s.reshape(-1, 1)
        yp_s_2d = yp_s.reshape(-1, 1)
        m_core = metrics_scaled_space(yt_s_2d, yp_s_2d)

        # percent metrics on ORIGINAL space for target i
        m_pct  = percent_metrics_original_space_single(yt_s, yp_s, scaler, j=i, clamp=True)

        rows.append({
            "Strategy": "A_t+1",
            "Model": name,
            "Target": var,
            "MSE": m_core["MSE"],
            "RMSE": m_core["RMSE"],
            "MAE": m_core["MAE"],
            "R2": m_core["R2"],
            "NRMSE": m_core["NRMSE"],
            "MAPE%": m_pct["MAPE%"],
            "sMAPE%": m_pct["sMAPE%"],
            "MAPE_masked%": m_pct["MAPE_masked%"],
        })

In [6]:
# ===============================
# 4) LOAD MODELS
# ===============================
# Base paper LSTM (t+1)
base_model = LSTM(
    input_dim=D,
    lstm_hidden_size=128,          # MUST match training
    num_lstm_layers=2,             # 2 layers in your checkpoint
    lstm_dropout=0.0,
    layer_units=[128, 64],         # MLP head like checkpoint
    num_outputs=T,
    matrix_rep=True,
    exogenous_dim=0
).to(DEVICE)
base_model.load_state_dict(torch.load(CKPT_BASE_T1, map_location=DEVICE), strict=True)
base_model.eval()

# Basic Multistep LSTM
multi_model = MultiStepLSTM(
    input_size=D, hidden_size=128, num_layers=1,
    output_size=T, forecast_steps=H
).to(DEVICE)
multi_model.load_state_dict(torch.load(CKPT_MULTI, map_location=DEVICE), strict=True)
multi_model.eval()

# Seq2Seq Multistep LSTM (full dataset)
s2s_model = Seq2SeqLSTM(
    input_size=D, hidden_size=64, output_size=T, forecast_steps=H, num_layers=1
).to(DEVICE)
s2s_model.load_state_dict(torch.load(CKPT_S2S_MULTI, map_location=DEVICE), strict=True)
s2s_model.eval()

# Seq2Seq Multistep LSTM with Clusters (cluster dataset)
D_c = X_te_c.shape[2]
s2s_cluster_model = Seq2SeqLSTM(
    input_size=D_c, hidden_size=64, output_size=T, forecast_steps=H, num_layers=1
).to(DEVICE)
s2s_cluster_model.load_state_dict(torch.load(CKPT_S2S_CLU, map_location=DEVICE), strict=True)
s2s_cluster_model.eval()

# Transformer multistep (full dataset)
transformer_model = TimeSeriesTransformer(
    input_size=D, d_model=128, nhead=4,
    num_encoder_layers=2, dim_feedforward=256, dropout=0.1,
    output_size=T, forecast_steps=H
).to(DEVICE)
transformer_model.load_state_dict(torch.load(CKPT_TRANS, map_location=DEVICE), strict=True)
transformer_model.eval()

TimeSeriesTransformer(
  (input_proj): Linear(in_features=6, out_features=128, bias=True)
  (enc_pos): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_pos): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): TransformerDecoder(
    (lay

In [7]:
# ===============================
# 5) PREDICT t+1 (SCALED space)
# ===============================
with torch.no_grad():
    xb_full = torch.tensor(X_test, dtype=torch.float32, device=DEVICE)     # [N, L, D]
    xb_clu  = torch.tensor(X_te_c, dtype=torch.float32, device=DEVICE)     # [Nc, L, Dc]

    # Base (t+1)
    base_t1_scaled = base_model(xb_full, device=DEVICE)                    # [N, 5]

    # Basic multistep -> pick t+1
    multi_all = multi_model(xb_full)                                       # [N, 6, 5]
    multi_t1_scaled = multi_all[:, 0, :]

    # Seq2Seq multistep (full)
    s2s_all = s2s_model(xb_full, teacher_forcing_ratio=0.0)
    s2s_t1_scaled = s2s_all[:, 0, :]

    # Seq2Seq multistep (cluster dataset)
    s2s_clu_all = s2s_cluster_model(xb_clu, teacher_forcing_ratio=0.0)
    s2s_clu_t1_scaled = s2s_clu_all[:, 0, :]

    # Transformer (full)
    trans_all = transformer_model(xb_full)
    trans_t1_scaled = trans_all[:, 0, :]

# to numpy
base_t1_scaled = base_t1_scaled.cpu().numpy()
multi_t1_scaled = multi_t1_scaled.cpu().numpy()
s2s_t1_scaled = s2s_t1_scaled.cpu().numpy()
s2s_clu_t1_scaled = s2s_clu_t1_scaled.cpu().numpy()
trans_t1_scaled = trans_t1_scaled.cpu().numpy()

In [8]:
# ===============================
# 6) STRATEGY A: BUILD COMPARISON TABLE (Strategy A, t+1)
#    Core metrics on SCALED; percentage metrics on ORIGINAL
# ===============================
rows = []

# Four "full" models use (X_test / y_test / y_scaler_full)
FULL_MODELS = [
    ("Base paper LSTM (t+1)", base_t1_scaled, y_test_t1_scaled_full, y_scaler_full),
    ("Basic Multistep LSTM (t+1)", multi_t1_scaled, y_test_t1_scaled_full, y_scaler_full),
    ("Seq2Seq Multistep LSTM (t+1)", s2s_t1_scaled, y_test_t1_scaled_full, y_scaler_full),
    ("Transformer model (t+1)", trans_t1_scaled, y_test_t1_scaled_full, y_scaler_full),
]

# Cluster model uses its own test & scaler
CLU_MODELS = [
    ("Seq2Seq Multistep LSTM with Clusters (t+1)", s2s_clu_t1_scaled, y_te_c_t1_scaled, y_scaler_c),
]

def add_rows_for_model(name, y_pred_scaled, y_true_scaled, scaler, targets):
    for i, var in enumerate(targets):
        # per-target arrays on SCALED space
        yt_s = y_true_scaled[:, i]
        yp_s = y_pred_scaled[:, i]

        # Core metrics on scaled space
        m_core = metrics_scaled_space(yt_s.reshape(-1, 1), yp_s.reshape(-1, 1))
        # Percent metrics on original space (inverse-transform one column)
        m_pct  = percent_metrics_original_space_single(yt_s, yp_s, scaler, j=i, clamp=True)

        rows.append({
            "Strategy": "A_t+1",
            "Model": name,
            "Target": var,
            "MSE": m_core["MSE"],
            "RMSE": m_core["RMSE"],
            "MAE": m_core["MAE"],
            "R2": m_core["R2"],
            "NRMSE": m_core["NRMSE"],
            "MAPE%": m_pct["MAPE%"],
            "sMAPE%": m_pct["sMAPE%"],
            "MAPE_masked%": m_pct["MAPE_masked%"],
        })

# Add rows for all models
for mdl_name, pred_s, ytrue_s, scaler in FULL_MODELS:
    add_rows_for_model(mdl_name, pred_s, ytrue_s, scaler, TARGETS)

for mdl_name, pred_s, ytrue_s, scaler in CLU_MODELS:
    add_rows_for_model(mdl_name, pred_s, ytrue_s, scaler, TARGETS)

df_t1 = pd.DataFrame(rows, columns=[
    "Strategy","Model","Target","MSE","RMSE","MAE","R2","NRMSE","MAPE%","sMAPE%","MAPE_masked%"
])
df_t1

,Strategy,Model,Target,MSE,RMSE,MAE,R2,NRMSE,MAPE%,sMAPE%,MAPE_masked%
0,A_t+1,Base paper LSTM (t+1),rnti_count,0.008045,0.089696,0.066063,0.485215,0.125351,2.548788e+01,24.742963,25.487880
1,A_t+1,Base paper LSTM (t+1),rb_down,0.008326,0.091249,0.043579,0.559291,0.091397,4.725889e+01,36.282234,47.258889
2,A_t+1,Base paper LSTM (t+1),rb_up,0.012164,0.110291,0.052190,0.610206,0.110291,2.718907e+03,123.955360,2097.916283
3,A_t+1,Base paper LSTM (t+1),down,0.007340,0.085676,0.050102,0.480701,0.085995,3.716739e+01,31.237519,37.167388
4,A_t+1,Base paper LSTM (t+1),up,0.010708,0.103480,0.045972,0.556588,0.103480,2.214842e+13,95.755641,1250.913328
5,A_t+1,Basic Multistep LSTM (t+1),rnti_count,0.006707,0.081893,0.062210,0.570880,0.114447,2.713494e+01,23.737117,27.134945
6,A_t+1,Basic Multistep LSTM (t+1),rb_down,0.008512,0.092262,0.047042,0.549452,0.092412,5.133693e+01,36.843247,51.336928
7,A_t+1,Basic Multistep LSTM (t+1),rb_up,0.013288,0.115274,0.056397,0.574183,0.115274,1.149287e+03,127.611300,1108.546220
8,A_t+1,Basic Multistep LSTM (t+1),down,0.007688,0.087681,0.053393,0.456111,0.088007,4.220614e+01,32.552472,42.206141
9,A_t+1,Basic Multistep LSTM (t+1),up,0.012111,0.110049,0.052907,0.498501,0.110049,5.336696e+02,104.555285,534.263324


In [9]:
def infer_target_positions_from_data(X_test, y_t1_scaled):
    """
    Heuristic: for each target (column in y_t1_scaled), find the input feature
    column in X (last time step of the window) with the highest absolute correlation.
    This often recovers the correct mapping when the same variables appear in X.
    """
    assert X_test.ndim == 3  # [N, L, D]
    assert y_t1_scaled.ndim == 2  # [N, T]
    N, L, D = X_test.shape
    T = y_t1_scaled.shape[1]

    X_last = X_test[:, -1, :]  # [N, D]
    pos = []
    used = set()
    for i in range(T):
        corrs = []
        yt = y_t1_scaled[:, i]
        yt = yt - yt.mean()
        yt_std = yt.std() + 1e-12
        for j in range(D):
            xj = X_last[:, j]
            xj = xj - xj.mean()
            xj_std = xj.std() + 1e-12
            # Pearson corr (robust to constant cols)
            corr = float(np.mean((xj / xj_std) * (yt / yt_std)))
            corrs.append(abs(corr))
        # greedily pick the best unused feature index
        for j in np.argsort(corrs)[::-1]:
            if j not in used:
                pos.append(int(j))
                used.add(int(j))
                break
    return pos

def validate_positions(pos_list, D, T):
    """
    Make sure positions are within [0, D-1] and length == T.
    If invalid, fallback to [0,1,...,T-1] (if T <= D).
    """
    ok = (
        len(pos_list) == T and
        all(0 <= p < D for p in pos_list) and
        len(set(pos_list)) == T
    )
    if ok:
        return pos_list, True
    # fallback
    if T <= D:
        fallback = list(range(T))
        print(f"[WARN] Invalid TARGET_POS_IN_X={pos_list} for D={D}. "
              f"Falling back to {fallback}. Verify this mapping!")
        return fallback, False
    else:
        raise ValueError(f"Cannot fallback: T={T} > D={D}.")

# ---- Build or infer the mapping ----
# If you already have y_test_t1_scaled_full (shape [N, T]) from your pipeline:
try:
    inferred = infer_target_positions_from_data(X_test, y_test_t1_scaled_full)
except NameError:
    # if your t+1 ground truth array is named differently, adjust here:
    inferred = infer_target_positions_from_data(X_test, y_test[:, 0, :])  # if y_test is [N,H,T] on scaled space

TARGET_POS_IN_X, _ = validate_positions(inferred, D=X_test.shape[2], T=y_test.shape[2])
print("Using TARGET_POS_IN_X =", TARGET_POS_IN_X)


Using TARGET_POS_IN_X = [0, 1, 2, 3, 4]


In [10]:
# ===============================
# STRATEGY B: Roll base LSTM to t+6 and compare step-by-step
# ===============================

# ---------- 1) Rollout base LSTM on FULL dataset ----------
def roll_base_lstm_to_horizon(base_model, X_init, steps, target_pos_in_x, device="cpu"):
    """
    base_model: LSTM that predicts next-step [N, T] on *scaled* space.
    X_init:     [N, L, D] test window (scaled).
    steps:      int (e.g., 6)
    target_pos_in_x: list of indices in X feature vector where the T targets live.
    """
    base_model.eval()
    x_win = torch.tensor(X_init, dtype=torch.float32, device=device)  # [N, L, D]
    outs = []
    with torch.no_grad():
        for _ in range(steps):
            y_next = base_model(x_win, device=device)  # [N, T] scaled
            outs.append(y_next.unsqueeze(1))           # [N,1,T]
            # update last row with predicted targets
            last_row = x_win[:, -1, :].clone()
            for k, pos in enumerate(target_pos_in_x):
                last_row[:, pos] = y_next[:, k]
            # shift window
            x_win = torch.cat([x_win[:, 1:, :], last_row.unsqueeze(1)], dim=1)
    return torch.cat(outs, dim=1).detach().cpu().numpy()  # [N, steps, T] scaled

# Roll base
base_rolled_scaled = roll_base_lstm_to_horizon(
    base_model, X_test, steps=H, target_pos_in_x=TARGET_POS_IN_X, device=DEVICE
)  # [N, H, T]

In [11]:
# ---------- 2) Get multistep predictions for FULL dataset ----------
with torch.no_grad():
    xb_full = torch.tensor(X_test, dtype=torch.float32, device=DEVICE)
    basic_multi_scaled = multi_model(xb_full).detach().cpu().numpy()                    # [N, H, T]
    s2s_scaled         = s2s_model(xb_full, teacher_forcing_ratio=0.0).detach().cpu().numpy()
    trans_scaled       = transformer_model(xb_full).detach().cpu().numpy()

# Ground truth (scaled)
y_true_full_scaled = y_test  # [N, H, T] scaled

# Pack full-dataset models
FULL_MODELS_STEPS = [
    ("Base LSTM (rolled t+1..t+6)", base_rolled_scaled, y_true_full_scaled, y_scaler_full),
    ("Basic Multistep LSTM",         basic_multi_scaled, y_true_full_scaled, y_scaler_full),
    ("Seq2Seq Multistep LSTM",       s2s_scaled,         y_true_full_scaled, y_scaler_full),
    ("Transformer model",            trans_scaled,       y_true_full_scaled, y_scaler_full),
]

# ---------- 3) Get multistep predictions for CLUSTER dataset (no base roll here) ----------
# 1) Predict (scaled)
with torch.no_grad():
    xb_clu = torch.tensor(X_te_c, dtype=torch.float32, device=DEVICE)
    # [Nc, H, T] on scaled space
    s2s_clu_scaled = s2s_cluster_model(xb_clu, teacher_forcing_ratio=0.0).detach().cpu().numpy()

# 2) Ground truth (scaled)
y_true_clu_scaled = y_te_c  # [Nc, H, T]

# 3) Pack for downstream evaluation code
CLU_MODELS_STEPS = [("Seq2Seq Multistep LSTM with Clusters", s2s_clu_scaled, y_true_clu_scaled, y_scaler_c),]

In [12]:
# ---------- 4) Evaluate per-step & overall ----------
def evaluate_multistep_models(models_steps, targets, strategy_tag="B"):
    """
    models_steps: list of tuples (name, y_pred_steps_scaled [N,H,T], y_true_steps_scaled [N,H,T], scaler)
    returns: (df_steps, df_overall)
    """
    rows_steps = []
    rows_over  = []

    for name, y_pred_s, y_true_s, scaler in models_steps:
        # ensure numpy
        if isinstance(y_pred_s, torch.Tensor):
            y_pred_s = y_pred_s.cpu().numpy()
        if isinstance(y_true_s, torch.Tensor):
            y_true_s = y_true_s.cpu().numpy()

        # Per step
        for step in range(H):
            for j, var in enumerate(targets):
                yt = y_true_s[:, step, j]  # [N] scaled
                yp = y_pred_s[:, step, j]  # [N] scaled

                # core on scaled
                m_core = metrics_scaled_space(yt.reshape(-1,1), yp.reshape(-1,1))
                # percent on original (inverse-transform column j)
                m_pct  = percent_metrics_original_space_single(yt, yp, scaler, j=j, clamp=True)

                rows_steps.append({
                    "Strategy": f"{strategy_tag}_t+{step+1}",
                    "Step": step+1,
                    "Model": name,
                    "Target": var,
                    "MSE": m_core["MSE"],
                    "RMSE": m_core["RMSE"],
                    "MAE": m_core["MAE"],
                    "R2": m_core["R2"],
                    "NRMSE": m_core["NRMSE"],
                    "MAPE%": m_pct["MAPE%"],
                    "sMAPE%": m_pct["sMAPE%"],
                    "MAPE_masked%": m_pct["MAPE_masked%"],
                })

        # Overall flattened across steps
        for j, var in enumerate(targets):
            yt_all = y_true_s[:, :, j].reshape(-1)  # [N*H]
            yp_all = y_pred_s[:, :, j].reshape(-1)

            m_core_all = metrics_scaled_space(yt_all.reshape(-1,1), yp_all.reshape(-1,1))
            m_pct_all  = percent_metrics_original_space_single(yt_all, yp_all, scaler, j=j, clamp=True)

            rows_over.append({
                "Strategy": f"{strategy_tag}_overall",
                "Model": name,
                "Target": var,
                "MSE": m_core_all["MSE"],
                "RMSE": m_core_all["RMSE"],
                "MAE": m_core_all["MAE"],
                "R2": m_core_all["R2"],
                "NRMSE": m_core_all["NRMSE"],
                "MAPE%": m_pct_all["MAPE%"],
                "sMAPE%": m_pct_all["sMAPE%"],
                "MAPE_masked%": m_pct_all["MAPE_masked%"],
            })

    df_steps = pd.DataFrame(rows_steps, columns=[
        "Strategy","Step","Model","Target",
        "MSE","RMSE","MAE","R2","NRMSE","MAPE%","sMAPE%","MAPE_masked%"
    ])
    df_over  = pd.DataFrame(rows_over, columns=[
        "Strategy","Model","Target",
        "MSE","RMSE","MAE","R2","NRMSE","MAPE%","sMAPE%","MAPE_masked%"
    ])
    return df_steps, df_over

# Evaluate FULL set (base rolled + multistep models)
df_B_steps_full, df_B_overall_full = evaluate_multistep_models(
    FULL_MODELS_STEPS, targets=TARGETS, strategy_tag="B(FULL)"
)

# Evaluate CLUSTER set (clustered model only)
df_B_steps_cluster, df_B_overall_cluster = evaluate_multistep_models(
    CLU_MODELS_STEPS, targets=TARGETS, strategy_tag="B(CLUSTER)"
)


In [13]:
df_B_steps_full

,Strategy,Step,Model,Target,MSE,RMSE,MAE,R2,NRMSE,MAPE%,sMAPE%,MAPE_masked%
0,B(FULL)_t+1,1,Base LSTM (rolled t+1..t+6),rnti_count,0.008045,0.089696,0.066063,0.485215,0.125351,2.548788e+01,24.742963,25.487880
1,B(FULL)_t+1,1,Base LSTM (rolled t+1..t+6),rb_down,0.008326,0.091249,0.043579,0.559291,0.091397,4.725889e+01,36.282234,47.258889
2,B(FULL)_t+1,1,Base LSTM (rolled t+1..t+6),rb_up,0.012164,0.110291,0.052190,0.610206,0.110291,2.718907e+03,123.955360,2097.916283
3,B(FULL)_t+1,1,Base LSTM (rolled t+1..t+6),down,0.007340,0.085676,0.050102,0.480701,0.085995,3.716739e+01,31.237519,37.167388
4,B(FULL)_t+1,1,Base LSTM (rolled t+1..t+6),up,0.010708,0.103480,0.045972,0.556588,0.103480,2.214842e+13,95.755641,1250.913328
...,...,...,...,...,...,...,...,...,...,...,...,...
115,B(FULL)_t+6,6,Transformer model,rnti_count,0.008373,0.091502,0.068258,0.464889,0.127875,2.820985e+01,25.350329,28.209849
116,B(FULL)_t+6,6,Transformer model,rb_down,0.013979,0.118234,0.062566,0.259387,0.118425,6.814866e+01,46.381172,68.148656
117,B(FULL)_t+6,6,Transformer model,rb_up,0.024121,0.155309,0.083074,0.224826,0.155309,9.040568e+02,165.974300,905.249964
118,B(FULL)_t+6,6,Transformer model,down,0.010237,0.101178,0.061748,0.275143,0.101555,4.607308e+01,36.714351,46.073084


In [14]:
df_B_overall_full

,Strategy,Model,Target,MSE,RMSE,MAE,R2,NRMSE,MAPE%,sMAPE%,MAPE_masked%
0,B(FULL)_overall,Base LSTM (rolled t+1..t+6),rnti_count,0.035469,0.188333,0.146515,-1.268102,0.263197,4.980920e+01,71.783886,49.809201
1,B(FULL)_overall,Base LSTM (rolled t+1..t+6),rb_down,0.019102,0.138209,0.069796,-0.011670,0.138433,6.031678e+01,64.432056,60.316784
2,B(FULL)_overall,Base LSTM (rolled t+1..t+6),rb_up,0.026200,0.161864,0.089242,0.159088,0.161864,5.902770e+03,150.109993,4405.889792
3,B(FULL)_overall,Base LSTM (rolled t+1..t+6),down,0.017395,0.131889,0.084010,-0.231383,0.132380,4.920035e+01,65.189550,49.200349
4,B(FULL)_overall,Base LSTM (rolled t+1..t+6),up,0.020193,0.142103,0.073382,0.161388,0.142103,4.202638e+13,126.791798,2397.080643
5,B(FULL)_overall,Basic Multistep LSTM,rnti_count,0.007651,0.087469,0.067382,0.510763,0.122239,3.056769e+01,25.787734,30.567688
6,B(FULL)_overall,Basic Multistep LSTM,rb_down,0.011023,0.104990,0.056390,0.416203,0.105160,6.593857e+01,44.627017,65.938571
7,B(FULL)_overall,Basic Multistep LSTM,rb_up,0.017975,0.134071,0.067151,0.423073,0.134071,1.452835e+03,132.419971,1423.208809
8,B(FULL)_overall,Basic Multistep LSTM,down,0.009058,0.095175,0.059343,0.358750,0.095530,4.869219e+01,36.543371,48.692190
9,B(FULL)_overall,Basic Multistep LSTM,up,0.015295,0.123672,0.062848,0.364817,0.123672,1.291440e+12,115.102816,845.745857


In [15]:
df_B_steps_cluster

,Strategy,Step,Model,Target,MSE,RMSE,MAE,R2,NRMSE,MAPE%,sMAPE%,MAPE_masked%
0,B(CLUSTER)_t+1,1,Seq2Seq Multistep LSTM with Clusters,rnti_count,0.006578,0.081103,0.060159,0.579123,0.113342,2.546613e+01,22.700099,25.466134
1,B(CLUSTER)_t+1,1,Seq2Seq Multistep LSTM with Clusters,rb_down,0.008320,0.091212,0.042843,0.559653,0.091359,4.245123e+01,33.904991,42.451233
2,B(CLUSTER)_t+1,1,Seq2Seq Multistep LSTM with Clusters,rb_up,0.012070,0.109865,0.048668,0.613210,0.109865,1.116393e+03,111.354024,922.361071
3,B(CLUSTER)_t+1,1,Seq2Seq Multistep LSTM with Clusters,down,0.007053,0.083981,0.048396,0.501046,0.084293,3.577251e+01,30.067656,35.772513
4,B(CLUSTER)_t+1,1,Seq2Seq Multistep LSTM with Clusters,up,0.010509,0.102514,0.046178,0.564829,0.102514,7.649607e+12,94.609674,705.398858
5,B(CLUSTER)_t+2,2,Seq2Seq Multistep LSTM with Clusters,rnti_count,0.007232,0.085041,0.063856,0.537377,0.118846,2.766098e+01,24.161699,27.660983
6,B(CLUSTER)_t+2,2,Seq2Seq Multistep LSTM with Clusters,rb_down,0.009311,0.096494,0.046777,0.507030,0.096650,4.908285e+01,37.194283,49.082848
7,B(CLUSTER)_t+2,2,Seq2Seq Multistep LSTM with Clusters,rb_up,0.014151,0.118958,0.054107,0.546151,0.118958,1.744133e+03,119.306079,1413.079040
8,B(CLUSTER)_t+2,2,Seq2Seq Multistep LSTM with Clusters,down,0.007697,0.087731,0.050923,0.455398,0.088058,3.838189e+01,31.742458,38.381892
9,B(CLUSTER)_t+2,2,Seq2Seq Multistep LSTM with Clusters,up,0.012285,0.110839,0.050256,0.490553,0.110839,1.154089e+13,100.822851,896.175286


In [16]:
df_B_overall_cluster

,Strategy,Model,Target,MSE,RMSE,MAE,R2,NRMSE,MAPE%,sMAPE%,MAPE_masked%
0,B(CLUSTER)_overall,Seq2Seq Multistep LSTM with Clusters,rnti_count,0.007655,0.087495,0.065804,0.510479,0.122275,2.878679e+01,24.891901,28.786791
1,B(CLUSTER)_overall,Seq2Seq Multistep LSTM with Clusters,rb_down,0.010641,0.103157,0.050989,0.436412,0.103324,5.464890e+01,40.256063,54.648896
2,B(CLUSTER)_overall,Seq2Seq Multistep LSTM with Clusters,rb_up,0.016619,0.128914,0.060093,0.466599,0.128914,1.725051e+03,123.114499,1451.936528
3,B(CLUSTER)_overall,Seq2Seq Multistep LSTM with Clusters,down,0.008558,0.092508,0.054063,0.394185,0.092853,4.091578e+01,33.413003,40.915782
4,B(CLUSTER)_overall,Seq2Seq Multistep LSTM with Clusters,up,0.014199,0.119159,0.055064,0.410333,0.119159,1.098793e+13,103.752390,941.065723


In [25]:
# import pandas as pd
# import numpy as np
# import torch
# from argparse import Namespace
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# # Add the parent directory of 'ml' to sys.path
# import sys, os
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# import warnings
# warnings.filterwarnings('ignore')


# from ml.utils.data_utils import prepare_dataset
# from ml.models.lstm import LSTM
# from ml.models.multi_step_lstm import MultiStepLSTM

In [26]:
# # ─────────────────────────────────────────────────────────────
# # 0) CONFIG
# args = Namespace(
#     data_path='../dataset/full_dataset.csv',
#     targets=['rnti_count', 'rb_down', 'rb_up', 'down', 'up'],
#     num_lags=10,
#     forecast_steps=6,
#     test_size=0.2,
#     ignore_cols=None,          # keep if you don't want to drop any
#     identifier='District',
#     nan_constant=0,
#     x_scaler='minmax',
#     y_scaler='minmax',
#     outlier_detection=True,
#     # Model hyperparams (must match what you used when training and saving the weights)
#     hidden_size=128,
#     num_layers=1,
#     device='cuda' if torch.cuda.is_available() else 'cpu',
#     seed=0
# )

# torch.manual_seed(args.seed)
# np.random.seed(args.seed)

In [27]:
# # Where are the 5 targets located inside X's feature dimension?
# # If your input feature vector is [rnti_count, rb_down, rb_up, down, up, <maybe_other_feature>],
# # the indices for the 5 targets are [0,1,2,3,4].
# TARGET_POS_IN_X = [0, 1, 2, 3, 4]   # <-- CHANGE IF NEEDED

# # Filenames of saved models
# BASE_MODEL_PATH   = "base_lstm_t1.pt"
# MULTISTEP_PATH    = "multi_step_lstm.pt"

In [28]:
# # ─────────────────────────────────────────────────────────────
# # 1) DATA
# X_train, y_train, X_test, y_test, x_scaler, y_scaler, id_train, id_test = prepare_dataset(args)
# # y_* is shape [N, 6, 5] (6 steps, 5 targets). For base LSTM evaluation we’ll use only t+1 labels.
# y_train_t1 = y_train[:, 0, :]      # [N, 5]
# y_test_t1  = y_test[:, 0, :]       # [N, 5]

# N, L, D = X_test.shape
# output_dim = len(args.targets)

In [29]:
# # Base paper LSTM (predicts t+1 only)
# base_model = LSTM(
#     input_dim=D,
#     lstm_hidden_size=args.hidden_size,      # must match the hidden size used when training
#     num_lstm_layers=2,                      # <-- checkpoint has 2 LSTM layers
#     lstm_dropout=0.0,                       # use what you trained with; dropout doesn't affect state_dict keys
#     layer_units=[128, 64],                  # <-- checkpoint has two hidden Linear layers before the output
#     num_outputs=len(args.targets),          # 5
#     matrix_rep=True,                        # your X is [batch, seq_len, feat]
#     exogenous_dim=0
# ).to(args.device)

# state = torch.load(BASE_MODEL_PATH, map_location=args.device)
# base_model.load_state_dict(state, strict=True)  # should load cleanly now
# base_model.eval()

In [30]:
# # Multi-step LSTM (direct 6-step prediction)
# multi_model = MultiStepLSTM(
#     input_size=D,
#     hidden_size=args.hidden_size,
#     num_layers=args.num_layers,
#     output_size=output_dim,
#     forecast_steps=args.forecast_steps
# ).to(args.device)

# multi_model.load_state_dict(torch.load(MULTISTEP_PATH, map_location=args.device))
# multi_model.eval()

In [31]:
# # ─────────────────────────────────────────────────────────────
# # 3) METRICS

# def safe_mape(y_true, y_pred, eps=1e-8):
#     denom = np.clip(np.abs(y_true), eps, None)
#     return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

# def eval_metrics(y_true, y_pred, name):
#     mse  = mean_squared_error(y_true, y_pred)
#     rmse = mean_squared_error(y_true, y_pred, squared=False)
#     mae  = mean_absolute_error(y_true, y_pred)
#     r2   = r2_score(y_true, y_pred)
#     nrmse = rmse / (y_true.max() - y_true.min() + 1e-8)
#     mape = safe_mape(y_true, y_pred)
#     print(f"\n{name}:")
#     print(f"  MSE   : {mse:.6f}")
#     print(f"  RMSE  : {rmse:.6f}")
#     print(f"  MAE   : {mae:.6f}")
#     print(f"  R²    : {r2:.6f}")
#     print(f"  NRMSE : {nrmse:.6f}")
#     print(f"  MAPE% : {mape:.3f}")
#     return dict(MSE=mse, RMSE=rmse, MAE=mae, R2=r2, NRMSE=nrmse, MAPE=mape)

In [32]:
# # ─────────────────────────────────────────────────────────────
# # 4) STRATEGY A: t+1 vs t+1
# with torch.no_grad():
#     xb = torch.tensor(X_test, dtype=torch.float32).to(args.device)
#     base_t1 = base_model(xb, device=args.device)         # [N, 5]
#     multi_all = multi_model(xb)                           # [N, 6, 5]
# base_t1  = base_t1.cpu().numpy()
# multi_t1 = multi_all[:, 0, :].cpu().numpy()

# print("\n===== Strategy A: Compare t+1 vs t+1 =====")
# for i, var in enumerate(args.targets):
#     # Ground truth t+1
#     y_true_t1 = y_test_t1[:, i]
#     # Base LSTM t+1
#     eval_metrics(y_true_t1, base_t1[:, i], name=f"Base t+1 — {var}")
#     # Multi-step LSTM t+1
#     eval_metrics(y_true_t1, multi_t1[:, i], name=f"Multi t+1 — {var}")

In [33]:
# # ─────────────────────────────────────────────────────────────
# # 5) STRATEGY B: Roll base LSTM to t+6 (recursive) and compare step-by-step

# def roll_base_lstm_to_horizon(base_model, X_init, steps, target_pos_in_x, device="cpu"):
#     """
#     base_model: LSTM that predicts next-step (t+1) targets, returns [N, 5]
#     X_init:     [N, L, D] initial window
#     steps:      number of recursive steps (e.g., 6)
#     target_pos_in_x: positions in input feature vector where the 5 targets live
#     """
#     base_model.eval()
#     x_win = torch.tensor(X_init, dtype=torch.float32, device=device)  # [N, L, D]
#     out_steps = []
#     with torch.no_grad():
#         for t in range(steps):
#             y_next = base_model(x_win, device=device)  # [N, 5]
#             out_steps.append(y_next.unsqueeze(1))      # [N,1,5]

#             # Build updated last row
#             last_row = x_win[:, -1, :].clone()         # [N, D]
#             for k, pos in enumerate(target_pos_in_x):
#                 last_row[:, pos] = y_next[:, k]

#             # Shift the window: drop oldest, append updated last_row
#             x_win = torch.cat([x_win[:, 1:, :], last_row.unsqueeze(1)], dim=1)

#     return torch.cat(out_steps, dim=1).cpu().numpy()     # [N, steps, 5]

# base_rolled = roll_base_lstm_to_horizon(
#     base_model, X_test, steps=args.forecast_steps, target_pos_in_x=TARGET_POS_IN_X, device=args.device
# )  # [N, 6, 5]

# # Multi-step predictions already computed above (multi_all)
# multi_preds = multi_all.cpu().numpy()  # [N, 6, 5]

# print("\n===== Strategy B: Step-by-step comparison (t+1…t+6) =====")
# for step in range(args.forecast_steps):
#     print(f"\n-- Step t+{step+1} --")
#     for i, var in enumerate(args.targets):
#         y_true_step = y_test[:, step, i]
#         eval_metrics(y_true_step, base_rolled[:, step, i], name=f"Base t+{step+1} — {var}")
#         eval_metrics(y_true_step, multi_preds[:, step, i], name=f"Multi t+{step+1} — {var}")

# print("\n-- Overall (flattened across steps) --")
# for i, var in enumerate(args.targets):
#     eval_metrics(y_test[:, :, i].ravel(), base_rolled[:, :, i].ravel(), name=f"Base t+1..t+6 — {var}")
#     eval_metrics(y_test[:, :, i].ravel(), multi_preds[:, :, i].ravel(), name=f"Multi t+1..t+6 — {var}")

# print("\n✅ Done. Compared base paper LSTM vs multi-step LSTM using:")
# print("   A) t+1 vs t+1, and")
# print("   B) Recursive rollout for base LSTM to t+6 vs direct multi-step predictions.")

In [34]:
# # import torch, numpy as np, pandas as pd
# # from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# # ---------- helpers ----------
# def _safe_mape(y_true, y_pred, eps=1e-8):
#     y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
#     denom = np.clip(np.abs(y_true), eps, None)
#     return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

# def eval_metrics(y_true, y_pred):
#     mse  = mean_squared_error(y_true, y_pred)
#     rmse = mean_squared_error(y_true, y_pred, squared=False)
#     mae  = mean_absolute_error(y_true, y_pred)
#     r2   = r2_score(y_true, y_pred)
#     nrmse = rmse / (np.max(y_true) - np.min(y_true) + 1e-8)
#     mape = _safe_mape(y_true, y_pred)
#     return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2, "NRMSE": nrmse, "MAPE%": mape}

# # ---------- STRATEGY A: t+1 vs t+1 → single dataframe ----------
# with torch.no_grad():
#     xb = torch.tensor(X_test, dtype=torch.float32).to(args.device)
#     base_t1  = base_model(xb, device=args.device)            # [N, 5]
#     multi_all = multi_model(xb)                               # [N, 6, 5]
# base_t1_np  = base_t1.detach().cpu().numpy()                 # [N, 5]
# multi_t1_np = multi_all[:, 0, :].detach().cpu().numpy()      # [N, 5]

# rows_A = []
# for i, var in enumerate(args.targets):
#     y_true = y_test[:, 0, i]          # ground-truth t+1
#     rows_A.append({"Strategy": "A_t+1", "Model": "Base LSTM (t+1)", "Target": var, **eval_metrics(y_true, base_t1_np[:, i])})
#     rows_A.append({"Strategy": "A_t+1", "Model": "Multi-step (t+1)", "Target": var, **eval_metrics(y_true, multi_t1_np[:, i])})

# df_strategy_A = pd.DataFrame(rows_A).reset_index(drop=True)

# # ---------- STRATEGY B: roll base to t+6 and compare step-by-step → two dataframes ----------
# def roll_base_lstm_to_horizon(base_model, X_init, steps, target_pos_in_x, device="cpu"):
#     base_model.eval()
#     x_win = torch.tensor(X_init, dtype=torch.float32, device=device)  # [N, L, D]
#     outs = []
#     with torch.no_grad():
#         for _ in range(steps):
#             y_next = base_model(x_win, device=device)                  # [N, 5]
#             outs.append(y_next.unsqueeze(1))
#             last_row = x_win[:, -1, :].clone()
#             for k, pos in enumerate(target_pos_in_x):
#                 last_row[:, pos] = y_next[:, k]
#             x_win = torch.cat([x_win[:, 1:, :], last_row.unsqueeze(1)], dim=1)
#     return torch.cat(outs, dim=1).detach().cpu().numpy()               # [N, steps, 5]

# # roll base and get multi-step predictions
# base_rolled = roll_base_lstm_to_horizon(
#     base_model, X_test, steps=args.forecast_steps, target_pos_in_x=TARGET_POS_IN_X, device=args.device
# )  # [N, 6, 5]
# multi_preds = multi_all.detach().cpu().numpy()                         # [N, 6, 5]

# # Per-step metrics table
# rows_B_steps = []
# for step in range(args.forecast_steps):
#     for i, var in enumerate(args.targets):
#         y_true = y_test[:, step, i]
#         rows_B_steps.append({
#             "Strategy": f"B_t+{step+1}",
#             "Step": step+1,
#             "Model": "Base LSTM (rolled)",
#             "Target": var,
#             **eval_metrics(y_true, base_rolled[:, step, i])
#         })
#         rows_B_steps.append({
#             "Strategy": f"B_t+{step+1}",
#             "Step": step+1,
#             "Model": "Multi-step",
#             "Target": var,
#             **eval_metrics(y_true, multi_preds[:, step, i])
#         })

# df_strategy_B_steps = pd.DataFrame(rows_B_steps).reset_index(drop=True)

# # Overall (flatten across steps) metrics table
# rows_B_overall = []
# for i, var in enumerate(args.targets):
#     y_true_all   = y_test[:, :, i].ravel()
#     base_all     = base_rolled[:, :, i].ravel()
#     multi_all_np = multi_preds[:, :, i].ravel()
#     rows_B_overall.append({"Strategy": "B_overall", "Model": "Base LSTM (t+1..t+6)", "Target": var, **eval_metrics(y_true_all, base_all)})
#     rows_B_overall.append({"Strategy": "B_overall", "Model": "Multi-step (t+1..t+6)", "Target": var, **eval_metrics(y_true_all, multi_all_np)})

# df_strategy_B_overall = pd.DataFrame(rows_B_overall).reset_index(drop=True)

# # # ---------- Show / Save ----------
# # print("\n=== Strategy A: t+1 vs t+1 (both models) ===")
# # print(df_strategy_A)

# # print("\n=== Strategy B (per step): base rolled vs multi-step ===")
# # print(df_strategy_B_steps)

# # print("\n=== Strategy B (overall flattened across steps) ===")
# # print(df_strategy_B_overall)

# # Optional: save to CSV
# # df_strategy_A.to_csv("compare_strategy_A_t1.csv", index=False)
# # df_strategy_B_steps.to_csv("compare_strategy_B_steps.csv", index=False)
# # df_strategy_B_overall.to_csv("compare_strategy_B_overall.csv", index=False)


In [35]:
# df_strategy_A

In [36]:
# df_strategy_B_steps

In [37]:
# df_strategy_B_overall